In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
backtest_all=pd.read_csv("./데이터/일일주식데이터.csv",encoding='cp949',index_col=0)
data = pd.read_csv("./데이터/5. label 지정 완료.csv", encoding='cp949')
feature = pd.read_csv("./데이터/8-2. Std feature.csv")
data

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,56.40,-5.90,-1.43,...,1.55,3472.17,12.72,35.86,128.51,19.56,89.82,10.18,-43.44,0.0
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-8.43,-4.12,7.73,...,0.57,269.31,0.47,2.89,10.67,1.66,1749.87,-1649.87,-2473.07,0.0
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,25.51,16.09,56.29,...,0.52,6994.90,12.51,77.91,289.40,34.29,71.28,28.72,4.23,0.0
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,11.52,74.30,6.37,...,0.80,7061.34,12.16,37.80,132.06,31.04,79.30,20.70,2.24,0.0
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,18.85,-5.37,11.04,...,1.22,4315.37,7.63,24.21,90.92,15.96,144.10,-44.10,-84.58,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,흥구석유(주),24060,2016-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-7.26,-5.61,52.46,...,2.20,13728.30,10.92,102.82,3801.25,6.86,44.46,55.54,43.99,0.0
9274,흥구석유(주),24060,2017-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,8.12,-0.56,9.41,...,1.78,14659.03,11.36,118.51,5704.66,9.22,43.35,56.65,43.02,0.0
9275,흥구석유(주),24060,2018-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,6.87,-0.36,-46.09,...,1.76,14287.12,10.80,118.81,4046.12,8.82,46.23,53.77,40.99,0.0
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-9.35,-7.00,96.40,...,2.18,11012.82,8.64,92.04,2603.45,5.61,62.14,37.86,27.35,0.0


In [3]:
data["상장일"]=pd.to_datetime(data['상장일'])
data["회계년도"] = pd.to_datetime(data["회계년도"])
data["투자년도"] = data["회계년도"].dt.year
data["투자년도"]

0       2011
1       2012
2       2013
3       2014
4       2015
        ... 
9273    2016
9274    2017
9275    2018
9276    2019
9277    2020
Name: 투자년도, Length: 9278, dtype: int64

In [4]:
fea = feature["logit_feature"].tolist()
fea

['매출액증가율',
 '유형자산증가율',
 '재고자산증가율',
 '비유동자산증가율',
 '유동자산증가율',
 '현금비율',
 '이자보상배율이자비용',
 '비유동비율',
 '자기자본구성비율',
 '자기자본배율',
 '매출채권회전률',
 '매출채권회전기간']

In [5]:
data.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명', '매출액증가율',
       '유형자산증가율', '재고자산증가율', '비유동자산증가율', '유동자산증가율', '자기자본증가율', '총자본증가율',
       '순이익증가율', '매출액총이익률', '매출액순이익률', '자기자본순이익률', '총자본사업이익률', '총자본순이익률',
       '1주당매출액', '1주당순이익', '유동비율', '당좌비율', '현금비율', '순운전자본비율', '부채비율',
       '이자보상배율이자비용', '비유동비율', '유동자산구성비율', '자기자본구성비율', '자기자본배율', '총자본회전률',
       '자기자본회전률', '비유동자산회전률', '재고자산회전률', '매출채권회전률', '매입채무회전률', '매입채무회전기간',
       '재고자산회전기간', '매출채권회전기간', '1회전기간', '운전자본회전률', '순운전자본회전률', '유형자산회전율',
       '유동자산회전률', '당좌자산회전률', '부가가치', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'label', '투자년도'],
      dtype='object')

In [6]:
data = data[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명','매출액증가율',
 '유형자산증가율',
 '재고자산증가율',
 '비유동자산증가율',
 '유동자산증가율',
 '현금비율',
 '이자보상배율이자비용',
 '비유동비율',
 '자기자본구성비율',
 '자기자본배율',
 '매출채권회전률',
 '매출채권회전기간','label','투자년도']]

In [7]:
data_2011 = data[data["투자년도"]==2011]
data_2011 

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,유동자산증가율,현금비율,이자보상배율이자비용,비유동비율,자기자본구성비율,자기자본배율,매출채권회전률,매출채권회전기간,label,투자년도
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,56.40,-5.90,-1.43,...,17.64,15.10,3.53,66.34,66.05,82.55,2.45,148.80,0.0,2011
10,(주)ES큐브,50120,2011-12-01,2002-04-25,NaN,5,도매 및 상품 중개업,21.63,27.82,27.49,...,3.83,21.35,-245.99,54.53,70.74,327.25,4.27,85.45,0.0,2011
20,(주)가비아,79940,2011-12-01,2005-10-19,NaN,5,출판업,30.07,11.47,-53.95,...,23.70,86.37,22.28,74.21,74.92,432.15,12.66,28.83,0.0,2011
46,(주)고영테크놀러지,98460,2011-12-01,2008-06-03,NaN,5,기타 기계 및 장비 제조업,18.06,17.31,55.18,...,24.16,42.81,22785.60,34.04,80.02,1608.46,3.74,97.64,0.0,2011
56,(주)골드앤에스,35290,2011-12-01,1999-12-07,NaN,5,출판업,-5.87,-24.47,-6.66,...,33.08,0.68,1.13,103.40,48.07,86.85,6.78,53.80,0.0,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9212,현대에이치티(주),39010,2011-12-01,2000-07-20,NaN,5,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",-18.79,-2.02,-17.70,...,-5.62,10.66,-171.85,73.46,45.97,550.26,2.75,132.61,0.0,2011
9222,현우산업(주),92300,2011-12-01,2007-10-24,NaN,5,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",7.58,8.40,-8.72,...,4.77,27.86,2.86,31.71,85.30,1035.88,1.62,225.24,0.0,2011
9236,화일약품(주),61250,2011-12-01,2002-04-18,NaN,5,의료용 물질 및 의약품 제조업,2.67,0.04,-13.39,...,-14.74,19.23,4.61,44.09,70.38,1240.16,2.31,157.82,0.0,2011
9252,휴림로봇(주),90710,2011-12-01,2006-12-05,NaN,5,기타 기계 및 장비 제조업,58.70,27.83,17.70,...,38.25,15.15,2.96,74.91,45.82,706.47,3.34,109.22,0.0,2011


In [8]:
data_2011.columns

Index(['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '소속코드', '산업명', '매출액증가율',
       '유형자산증가율', '재고자산증가율', '비유동자산증가율', '유동자산증가율', '현금비율', '이자보상배율이자비용',
       '비유동비율', '자기자본구성비율', '자기자본배율', '매출채권회전률', '매출채권회전기간', 'label', '투자년도'],
      dtype='object')